In [ ]:
# !pip install -U --quiet langgraph bitsandbytes

In [ ]:
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain_core.runnables import Runnable
from langchain.agents import AgentExecutor, initialize_agent
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents import create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import Tool

In [ ]:
# Step 1: Define the tool
@tool
def calculator(expression: str) -> str:
    """Evaluates a basic mathematical expression like 2+3*4"""
    try:
        return str(eval(expression, {"__builtins__": {}}, {}))
    except Exception as e:
        print(e)
        return f"Error: {e}"

tools = [Tool.from_function(calculator, calculator.name, calculator.description)]

In [ ]:
# Step 2: Load HuggingFace LLM (e.g., Mistral)
model_id = "deepseek-ai/deepseek-math-7b-rl"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto")

hf_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
llm = HuggingFacePipeline(pipeline=hf_pipe)

In [ ]:
# Step 3: Build the agent with OpenAI function calling style
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a expert mechanical engineer advisor. Use the calculator tool when needed."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

# agent_runnable: Runnable = create_openai_functions_agent(llm, tools, prompt)
# agent_executor = AgentExecutor(agent=agent_runnable, tools=tools, verbose=True)
tools=[calculator]
# llm.bind_tools(tools)
agent=initialize_agent(
    tools=tools,
    llm=llm,
    agent="conversational-react-description",
    verbose=True
)

In [ ]:
inp="What is the force exerted by a 10kg object due to gravity?"
agent.invoke({"input":inp,"agent_scratchpad":[],"chat_history":[]})

In [ ]:

# Step 4: LangGraph definition

# Define state
from typing import TypedDict, List

class AgentState(TypedDict):
    input: str
    chat_history: List
    agent_scratchpad: List
    output: str

# Agent node
def agent_node(state: AgentState) -> AgentState:
    result = agent.invoke({
        "input": state["input"],
        "chat_history": state["chat_history"],
        "agent_scratchpad": state["agent_scratchpad"]
    })
    return {
        "input": state["input"],
        "chat_history": state["chat_history"] + [HumanMessage(content=state["input"]), AIMessage(content=result["output"])],
        "agent_scratchpad": [],
        "output": result["output"]
    }

# Build the graph
graph = StateGraph(AgentState)
graph.add_node("agent", agent_node)
graph.set_entry_point("agent")
graph.set_finish_point("agent")  # End after one full run (you can loop with conditions too)

# Compile the graph
runnable_graph = graph.compile()

# Run it
input_question = "If I invest 3000 at 8% and 5000 at 12%, what is my total interest?"
result = runnable_graph.invoke({
    "input": input_question,
    "chat_history": [],
    "agent_scratchpad": [],
    "output": ""
})

print("\nFinal Answer:", result["output"])